#Building a song recommender


#Fire up GraphLab Create

In [1]:
import graphlab

#Load music data

In [2]:
song_data = graphlab.SFrame('song_data.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to yz2861@columbia.edu and will expire on November 11, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-2158 - Server binary: /Users/ZhangYunyan/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1447428017.log
[INFO] GraphLab Server Version: 1.6.1


#Explore data

Music data shows how many times a user listened to a song, as well as the details of the song.

In [5]:
song_data.head()

user_id,song_id,listen_count,title,artist
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Kanye West
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Héroes del Silencio
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,Lady GaGa
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Thievery Corporationfeat. Emiliana Torrini ...


##Showing the most popular songs in the dataset

In [3]:
graphlab.canvas.set_target('ipynb')

In [ ]:
song_data['song'].show()

In [9]:
print len(song_data)

1116609


In [19]:
print len(song_data[song_data['artist']=='Kanye West'])
print len(song_data[song_data['artist']=='Kanye West']['user_id'].unique())
print len(song_data[song_data['artist']=='Foo Fighters']['user_id'].unique())
print len(song_data[song_data['artist']=='Taylor Swift']['user_id'].unique())
print len(song_data[song_data['artist']=='Lady GaGa']['user_id'].unique())
#print len(usersk)

3775
2522
2055
3246
2928


##Count number of unique users in the dataset

In [10]:
users = song_data['user_id'].unique()

In [11]:
len(users)

66346

In [20]:
t=song_data.groupby(key_columns='artist', operations={'total_count': 
                                                    graphlab.aggregate.SUM('listen_count')})
t.sort('total_count',ascending=False)

artist,total_count
Kings Of Leon,43218
Dwight Yoakam,40619
Björk,38889
Coldplay,35362
Florence + The Machine,33387
Justin Bieber,29715
Alliance Ethnik,26689
OneRepublic,25754
Train,25402
The Black Keys,22184


In [31]:
t.sort('total_count',ascending=False).tail()

artist,total_count
Jody Bernal,38
Aneta Langerova,38
Kanye West / Talib Kweli/ Q-Tip / Common / ...,38
Nâdiya,36
harvey summers,31
Boggle Karaoke,30
Diplo,30
Beyoncé feat. Bun B andSlim Thug ...,26
Reel Feelings,24
William Tabbert,14


#Create a song recommender

In [21]:
train_data,test_data = song_data.random_split(.8,seed=0)

##Simple popularity-based recommender

In [ ]:
popularity_model = graphlab.popularity_recommender.create(train_data,
                                                         user_id='user_id',
                                                         item_id='song')

###Use the popularity model to make some predictions

A popularity model makes the same prediction for all users, so provides no personalization.

In [ ]:
popularity_model.recommend(users=[users[0]])

In [ ]:
popularity_model.recommend(users=[users[1]])

##Build a song recommender with personalization

We now create a model that allows us to make personalized recommendations to each user. 

In [22]:
personalized_model = graphlab.item_similarity_recommender.create(train_data,
                                                                user_id='user_id',
                                                                item_id='song')

PROGRESS: Recsys training: model = item_similarity
PROGRESS: Warning: Ignoring columns song_id, listen_count, title, artist;
PROGRESS:     To use one of these as a target column, set target = <column_name>
PROGRESS:     and use a method that allows the use of a target.
PROGRESS: Preparing data set.
PROGRESS:     Data has 893580 observations with 66085 users and 9952 items.
PROGRESS:     Data prepared in: 1.32509s
PROGRESS: Computing item similarity statistics:
PROGRESS: Computing most similar items for 9952 items:
PROGRESS: +-----------------+-----------------+
PROGRESS: | Number of items | Elapsed Time    |
PROGRESS: +-----------------+-----------------+
PROGRESS: | 1000            | 2.12157         |
PROGRESS: | 2000            | 2.2032          |
PROGRESS: | 3000            | 2.27553         |
PROGRESS: | 4000            | 2.34493         |
PROGRESS: | 5000            | 2.41912         |
PROGRESS: | 6000            | 2.48521         |
PROGRESS: | 7000            | 2.55104         |


In [23]:
subset_test_users = test_data['user_id'].unique()[0:10000]

In [24]:
personalized_model.recommend(subset_test_users,k=1)

PROGRESS: recommendations finished on 1000/10000 queries. users per second: 1021.65
PROGRESS: recommendations finished on 2000/10000 queries. users per second: 1114.04
PROGRESS: recommendations finished on 3000/10000 queries. users per second: 1106.03
PROGRESS: recommendations finished on 4000/10000 queries. users per second: 1125.91
PROGRESS: recommendations finished on 5000/10000 queries. users per second: 1122.44
PROGRESS: recommendations finished on 6000/10000 queries. users per second: 1131.94
PROGRESS: recommendations finished on 7000/10000 queries. users per second: 1142.58
PROGRESS: recommendations finished on 8000/10000 queries. users per second: 1131.07
PROGRESS: recommendations finished on 9000/10000 queries. users per second: 1106.81
PROGRESS: recommendations finished on 10000/10000 queries. users per second: 1112.06


user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Cuando Pase El Temblor -Soda Stereo ...,0.0194504525792,1
c067c22072a17d33310d7223d7b79f819e48cf42 ...,Grind With Me (ExplicitVersion) - Pretty Ricky ...,0.0459424433009,1
f6c596a519698c97f1591ad89f540d76f6a04f1a ...,Hey_ Soul Sister - Train,0.024906818152,1
696787172dd3f5169dc94deef97e427cee86147d ...,Senza Una Donna (WithoutA Woman) - Zucchero / ...,0.0170265780731,1
3a7111f4cdf3c5a85fd4053e3cc2333562e1e0cb ...,Heartbreak Warfare - JohnMayer ...,0.0320719577493,1
532e98155cbfd1e1a474a28ed96e59e50f7c5baf ...,Jive Talkin' (AlbumVersion) - Bee Gees ...,0.0118288659232,1
ee43b175ed753b2e2bce806c903d4661ad351a91 ...,Ricordati Di Noi -Valerio Scanu ...,0.0305171277997,1
e372c27f6cb071518ae500589ae02c126954c148 ...,Fall Out - The Police,0.0819672131148,1
83b1428917b47a6b130ed471b09033820be78a8c ...,Clocks - Coldplay,0.0440505289698,1
39487deef9345b1e22881245cabf4e7c53b6cf6e ...,Black Mirror - ArcadeFire ...,0.0417737699321,1


In [34]:
personalized_model.recommend(subset_test_users,k=1).groupby(key_columns='song', operations={'count': graphlab.aggregate.COUNT()}).sort('count',ascending=False)

PROGRESS: recommendations finished on 1000/10000 queries. users per second: 1086.03
PROGRESS: recommendations finished on 2000/10000 queries. users per second: 1127.7
PROGRESS: recommendations finished on 3000/10000 queries. users per second: 1152.77
PROGRESS: recommendations finished on 4000/10000 queries. users per second: 1178.77
PROGRESS: recommendations finished on 5000/10000 queries. users per second: 1202.77
PROGRESS: recommendations finished on 6000/10000 queries. users per second: 1210.01
PROGRESS: recommendations finished on 7000/10000 queries. users per second: 1199.67
PROGRESS: recommendations finished on 8000/10000 queries. users per second: 1199.56
PROGRESS: recommendations finished on 9000/10000 queries. users per second: 1206.92
PROGRESS: recommendations finished on 10000/10000 queries. users per second: 1214.83


song,count
Undo - Björk,431
Secrets - OneRepublic,388
Revelry - Kings Of Leon,232
You're The One - DwightYoakam ...,170
Fireflies - CharttraxxKaraoke ...,122
Hey_ Soul Sister - Train,106
Horn Concerto No. 4 in Eflat K495: II. Romance ...,99
Sehr kosmisch - Harmonia,71
OMG - Usher featuringwill.i.am ...,57
Dog Days Are Over (RadioEdit) - Florence + The ...,53


###Applying the personalized model to make song recommendations

As you can see, different users get different recommendations now.

In [ ]:
personalized_model.recommend(users=[users[0]])

In [ ]:
personalized_model.recommend(users=[users[1]])

###We can also apply the model to find similar songs to any song in the dataset

In [ ]:
personalized_model.get_similar_items(['With Or Without You - U2'])

In [ ]:
personalized_model.get_similar_items(['Chan Chan (Live) - Buena Vista Social Club'])

#Quantitative comparison between the models

We now formally compare the popularity and the personalized models using precision-recall curves. 

In [ ]:
if graphlab.version[:3] >= "1.6":
    model_performance = graphlab.compare(test_data, [popularity_model, personalized_model], user_sample=0.05)
    graphlab.show_comparison(model_performance,[popularity_model, personalized_model])
else:
    %matplotlib inline
    model_performance = graphlab.recommender.util.compare_models(test_data, [popularity_model, personalized_model], user_sample=.05)

The curve shows that the personalized model provides much better performance. 